# Molecular dynamics simulation using the Lennard-Jones potential the velocity-Verlet integrator and periodic boundary conditions

### Janos Revesz
### 19111202

The following notebook contains a simple simulation of how a system of atoms behave when the only force acting on them is from the Lennard-Jones potential. 
$$ V_{LJ}= 4 \epsilon \left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right]$$

The force acting on the particle is:
$$F=-\nabla V_{LJ}$$
  
$$F=-24\epsilon \bf{r} \left[\left(\frac{\sigma^{12}}{r^{13}}\right)-\left(\frac{\sigma^{6}}{r^{7}}\right)\right]$$

We are simulating 64 Neon particles placed in a box evenly on a 4*4*4 grid at a temperature of T=50K. The simulation is ran for 10.000 steps where dt=1e-15. So the total length of the simulation is 1e-11. 

In [ ]:
# Appropriate imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

### 1. Set up the force function

In [ ]:
# the function is from the in class solutions by Professor David Bowler
def find_forces_LJ(pos,N,sigma,epsilon,boxlen):
    """Evaluate the potential energy and force acting on each particle, return the 
        forces in an array with N dimensions where each element is the sum of 
        the forces on the Nth particle. Also return the total potential energy of 
        the system.
        
        Input
        
        pos: N*3 dimensional array, contains the position of each particle
        N: number of particles
        sigma: the distance at which the particle-particle potential energy is 0
        epsilon: characterises the depth of the potential well
        
        Output
        
        energy: the total potential energy of the system: calculated for every pair of 
        particles
        force: N*3 dimensional array, each 3D subarray corresponds to the total 
        forces acting on the Nth particle
        """
    force = np.zeros((N,3))
    energy = 0.0
    sigma6 = sigma**6
    sigma12 = sigma6*sigma6
    for i in range(N-1):
        for j in range(i+1,N):
            # vector between the ith and jth particle
            dr = pos[j]-pos[i]
            # esure that we interact with the closest particle through periodic boundaries
            dr -= boxlen*np.rint(dr/boxlen)
            dr2 = np.sum(dr*dr)
            dr2i = 1.0/dr2
            dr6i = dr2i*dr2i*dr2i
            # force between the two particles
            fij= 24.0*epsilon*dr*dr2i*dr6i*(sigma6 - 2.0*sigma12*dr6i)
            force[i] += fij
            force[j] -= fij
            # potential energy between the two particles
            energy += 4.0*epsilon*dr6i*(sigma12*dr6i - sigma6)
    return energy, force

### 2. Set up the lattice and the simulation parameters



In [ ]:
# Initialisation
Nsteps = 10000
kB = 1.38e-23         # J/K
sigma = 0.275e-9      # 0.275nm
epsilon = 36*kB       # 36kT
mass = 20.2*1.673e-27 # kg
Ncells = 4
boxlen = 1.1 * 2**(1/6) * sigma * Ncells 
Npart = 64
rbox3 = np.zeros([Nsteps,Npart,3])
vbox3 = np.zeros([Nsteps,Npart,3])
count = 0
for i in range(Ncells):
    for j in range(Ncells):
        for k in range(Ncells):
            rbox3[0,count] = (boxlen/Ncells) * np.array([i, j, k])
            count += 1

In [ ]:
# Plot
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(rbox3[0,:,0],rbox3[0,:,1],rbox3[0,:,2],c = 'orange')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

### 3. Initialise velocities

In [ ]:
# Initialise velocities
initT  = 50          # K
# The initial distribution can have an arbitrary scaling
vbox3[0] = np.random.normal(size=(Npart,3))
# These line rescales the velocities
v_scale = np.sqrt(3*Npart*kB*initT/(mass*np.sum(vbox3[0]*vbox3[0])))
vbox3[0] *= v_scale
energy = np.zeros(Nsteps)
temp = np.zeros(Nsteps)
e, f = find_forces_LJ(rbox3[0],Npart,sigma,epsilon,boxlen)
energy[0] = e
temp[0] = mass*np.sum(vbox3[0]*vbox3[0])/(3*Npart*kB)
acc = f/mass
dt = 1e-15 # s 
MSD = np.zeros(Nsteps)

### 4. Run simulation


In [ ]:
for step in range(1,Nsteps):
    rbox3[step] = rbox3[step-1] + vbox3[step-1]*dt + acc/2*dt**2
    # Periodic boundaries
    rbox3[step] = rbox3[step]%boxlen
    # Verlet integrator
    energy[step], f = find_forces_LJ(rbox3[step],Npart,sigma,epsilon,boxlen)
    acc_next = f/mass
    vbox3[step] = vbox3[step-1] + (acc_next + acc)/2*dt
    temp[step] = mass*np.sum(vbox3[step]**2)/(3*Npart*kB)
    acc = acc_next
    # Calculate mean-squared displacement
    MSD[step-1] = np.sum((rbox3[step]-rbox3[0])**2)
MSD /= Npart

In [ ]:
# Plotting the total energy, kinetic energy and potential energy
# The code is from Prof. David Bowlers exercise solutions
timeaxis = np.linspace(0,Nsteps*dt - dt,Nsteps)
ke = 0.5*mass*np.sum(vbox3*vbox3,axis=(1,2))
plt.plot(timeaxis,ke/(Npart*epsilon),label='Kinetic')
plt.plot(timeaxis,energy/(Npart*epsilon),label='Potential')
plt.plot(timeaxis,(ke+energy)/(Npart*epsilon),label='Total')
plt.title("Energy per particle")
plt.xlabel("Time in s")
plt.ylabel("Energy/epsilon")
plt.legend()
plt.show()

The total energy of the system remains constant in time. The kinetic energy and potential energy oscillates around a constant value.

In [ ]:
time0 = dt*np.arange(1,Nsteps)
plt.plot(time0,MSD[:-1])
plt.title("Mean squared displacement of particles")
plt.xlabel("Time in s")
plt.ylabel("D(t)")
plt.show()

In [ ]:
plt.plot(time0[2000:],MSD[2000:-1]/(6*time0[2000:]))
plt.title("Mean squared displacement divided be 6t")
plt.xlabel("Time in s")
plt.ylabel("D(t)/(6t)")
plt.show()

The plot seems to converge to a constant value around 1e-8.

### 5. Calculate the auto-correlation function

In [ ]:
Int = np.zeros(Nsteps)
for step in range(1,Nsteps):
    Int[step] = Int[step-1] + np.sum(vbox3[step]*vbox3[0])*dt
plt.plot(time0, 1/(3*Npart)*Int[1:], label="Int(t)")
plt.plot(time0[100:],MSD[100:-1]/(6*time0[100:])/6, label="D(t)/36t")
plt.title("Auto-correlation function")
plt.xlabel("Time in s")
plt.legend()
plt.show()

The two functions do not seem to converge to the same  point but they do get fairly close on the scale of 0.15e-7. 

## Conclusion
Both the auto-correlation function and the mean squared displacement over 6t converge over time, but not to the same value, at least for 10000 timesteps. Calculating the Integral and mean squared displacement is negligable to the time it takes to apply the Verlet integrator.


  Running the simulation for longer timesteps should lead to a linear increase in time while running the simulation with more particles should lead to a parabolic increase of time taken. This is because for n particles there are n(n-1)/2 pairs and the force has to be calculated each time for all of these pairs.